In [ ]:

print("import required packages")
import cv2
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
#from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

print("NEXT:Initialize image data generator with rescaling")
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)
print("DONE:Initialize image data generator with rescaling")

print("NEXT:Preprocess all train images")
train_generator = train_data_gen.flow_from_directory(
        '/content/drive/My Drive/EMOTİONS/train',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')
print("DONE:Preprocess all test images")


print("NEXT:Preprocess all test images")
validation_generator = validation_data_gen.flow_from_directory(
        '/content/drive/My Drive/EMOTİONS/test',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')
print("DONE:Preprocess all train images")


print("NEXT:create model structure")
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
print("DONE:create model structure")


emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

cv2.ocl.setUseOpenCL(False)

emotion_model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

print("NEXT:Train the neural network/model")
emotion_model_info = emotion_model.fit_generator(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=5,
        validation_data=validation_generator,
        validation_steps=7178 // 64)
print("DONE:Train the neural network/model")


print("NEXT: save model structure in jason file")
model_json = emotion_model.to_json()
with open("emotion_model.json", "w") as json_file:
    json_file.write(model_json)
print("DONE: save model structure in jason file")


print("NEXT: save trained model weight in .h5 file")
emotion_model.save_weights('emotion_model.h5')
print("DONE: save trained model weight in .h5 file")



import required packages
NEXT:Initialize image data generator with rescaling
DONE:Initialize image data generator with rescaling
NEXT:Preprocess all train images
Found 28836 images belonging to 7 classes.
DONE:Preprocess all test images
NEXT:Preprocess all test images
Found 7178 images belonging to 7 classes.
DONE:Preprocess all train images
NEXT:create model structure
DONE:create model structure
NEXT:Train the neural network/model


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:66: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/5
375/448 [========================>.....] - ETA: 33:39 - loss: 1.7613 - accuracy: 0.2754

In [ ]:
import cv2
import numpy as np
from keras.models import model_from_json


emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

# load json and create model
json_file = open('/content/emotion_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)

# load weights into new model
emotion_model.load_weights("/content/emotion_model.h5")
print("Loaded model from disk")

# start the webcam feed
cap = cv2.VideoCapture("/content/drive/MyDrive/EMOTİONS/emotions.HEIC")
cv2.imread(cap)



while True:
    # Find haar cascade to draw bounding box around face
    ret, frame = cap.read()
    
    if not ret:
        break
    face_detector = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # detect faces available on camera
    num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    # take each face available on the camera and Preprocess it
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)

        # predict the emotions
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    cv2.imshow('Emotion Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
